# Run OpenQuake scenarios sensitivity analysis

## Configuration and user input

In [5]:
# Import dependencies
import os, glob
import oq_run_many as oq
from oq_parse_gmfs import oq_get_gmfs, write_gmf_csv


# Event name
event = '20140000_Sequence_Kefalonia/20140126_M6.1_Kefalonia'

# Indicate ruptures to be considered in calculations
ruptures_to_include = ['USGS', 'GCMT','EFEHR','ESM']

# Ground Motion Models or GMPE's to include.
gmm_sources_to_include = ['GEM', 'USGS']


## Folders and files

In [6]:
# Find all rupture files
all_rups = glob.glob(os.path.join('..', '*', event,
                                  'Rupture',
                                  'earthquake_rupture_model_*'))

# Verify that ruptures exist in folder
ruptures = []
for author in ruptures_to_include:
    exist = False 
    for rup in all_rups:
        if author in rup:
            exist = True
            ruptures.append(rup)
    if not exist:
        raise ValueError(f'No rupture found for author {author}!')

# OQ rupture paths
oq_rups = [f'../Rupture/{os.path.basename(rup)}' for rup in ruptures]

# OQ gmlt paths
oq_gmlt = [f'gmpe_logic_tree_{gmlt}.xml' for gmlt in gmm_sources_to_include]

# Event folder
country = ruptures[0].split(os.sep)[1]
root_folder = ruptures[0][:ruptures[0].find('Rupture')]
oq_folder = os.path.join(root_folder, 'OpenQuake_gmfs')
oq_sens = os.path.join(oq_folder, 'Sensitivity')

# Create Sensitivity folder
if os.path.isdir(oq_sens):
    print('Sensitivity folder already exists\n')
else:
    os.mkdir(oq_sens)
    print('Sensitivity folder created\n')


print('GMLT paths for OQ:', oq_gmlt)
print('Rupture paths for OQ:')
oq_rups


Sensitivity folder created

GMLT paths for OQ: ['gmpe_logic_tree_GEM.xml', 'gmpe_logic_tree_USGS.xml']
Rupture paths for OQ:


['../Rupture/earthquake_rupture_model_USGS.xml',
 '../Rupture/earthquake_rupture_model_GCMT.xml',
 '../Rupture/earthquake_rupture_model_EFEHR.xml']

## Run calculation without conditioning the GMFs

### Stations: None

In [7]:
# Job.ini to be use in the sensitivity analysis
job_ini = os.path.join(oq_folder, 'job_stations_none.ini')

# Run OpenQuake calculations
jobs = oq.main(job_ini, oq_rups, oq_gmlt, concurrent_jobs=1)

# For each calculation, save gmfs in csv (only for PGA)
for job in jobs:
    calc_id = job.calc_id
    df, txt = oq_get_gmfs(calc_id)
    out_df = write_gmf_csv(df, 'PGA', comment=txt, 
                           out_path=oq_sens, suffix=calc_id)

[2023-06-07 11:18:53 #13 INFO] Memory occupation 82%, the user should free some memory
[2023-06-07 11:18:58 #13 INFO] Memory occupation 82%, the user should free some memory
[2023-06-07 11:19:03 #13 INFO] Memory occupation 82%, the user should free some memory
[2023-06-07 11:19:08 #13 INFO] Memory occupation 81%, the user should free some memory
[2023-06-07 11:19:13 #13 INFO] Memory occupation 81%, the user should free some memory
[2023-06-07 11:19:18 #13 INFO] Memory occupation 81%, the user should free some memory
[2023-06-07 11:19:23 #13 INFO] Memory occupation 82%, the user should free some memory
[2023-06-07 11:19:28 #13 INFO] Memory occupation 81%, the user should free some memory
[2023-06-07 11:19:33 #13 INFO] Memory occupation 81%, the user should free some memory
[2023-06-07 11:19:38 #13 INFO] Memory occupation 81%, the user should free some memory
[2023-06-07 11:19:44 #13 INFO] chago@MacBook-Pro-de-Santiago.local running /Users/chago/GEM/ecd/Greece/20151117_M6.5_Lefkada/OpenQ

  id | name
  49 | Events
  50 | Full Report
  51 | Ground Motion Fields
  52 | Realizations


[2023-06-07 11:19:47 #14 INFO] chago@MacBook-Pro-de-Santiago.local running /Users/chago/GEM/ecd/Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/job_stations_none.ini [--hc=None]
[2023-06-07 11:19:47 #14 INFO] Using engine version 3.17.0-git60e4ac41e1
[2023-06-07 11:19:47 #14 WARNING] Using 8 cores on MacBook-Pro-de-Santiago.local
[2023-06-07 11:19:47 #14 INFO] Running EventBasedCalculator with concurrent_tasks = 16
[2023-06-07 11:19:47 #14 INFO] Checksum of the inputs: 3257721805 (total size 215.2 KB)
[2023-06-07 11:19:47 #14 WARNING] Entering SLOW MODE. You have 1.9 GB available, but the engine would like at least 0.5 GB per core, i.e. 4.0 GB: https://github.com/gem/oq-engine/blob/master/doc/faq.md
[2023-06-07 11:19:47 #14 INFO] Extracting the hazard sites from the site model
/Users/chago/openquake/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column geology - only the converter will be used.
  return func(*args, *

  id | name
  53 | Events
  54 | Full Report
  55 | Ground Motion Fields
  56 | Realizations


[2023-06-07 11:19:49 #15 INFO] chago@MacBook-Pro-de-Santiago.local running /Users/chago/GEM/ecd/Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/job_stations_none.ini [--hc=None]
[2023-06-07 11:19:49 #15 INFO] Using engine version 3.17.0-git60e4ac41e1
[2023-06-07 11:19:49 #15 WARNING] Using 8 cores on MacBook-Pro-de-Santiago.local
[2023-06-07 11:19:49 #15 INFO] Running EventBasedCalculator with concurrent_tasks = 16
[2023-06-07 11:19:49 #15 INFO] Checksum of the inputs: 1328145222 (total size 215.19 KB)
[2023-06-07 11:19:49 #15 WARNING] Entering SLOW MODE. You have 1.9 GB available, but the engine would like at least 0.5 GB per core, i.e. 4.0 GB: https://github.com/gem/oq-engine/blob/master/doc/faq.md
[2023-06-07 11:19:49 #15 INFO] Extracting the hazard sites from the site model
/Users/chago/openquake/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column geology - only the converter will be used.
  return func(*args, 

...

  id | name
  61 | Events
  62 | Full Report
  63 | Ground Motion Fields
  64 | Realizations


[2023-06-07 11:19:54 #17 INFO] chago@MacBook-Pro-de-Santiago.local running /Users/chago/GEM/ecd/Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/job_stations_none.ini [--hc=None]
[2023-06-07 11:19:54 #17 INFO] Using engine version 3.17.0-git60e4ac41e1
[2023-06-07 11:19:54 #17 WARNING] Using 8 cores on MacBook-Pro-de-Santiago.local
[2023-06-07 11:19:54 #17 INFO] Running EventBasedCalculator with concurrent_tasks = 16
[2023-06-07 11:19:54 #17 INFO] Checksum of the inputs: 3191985813 (total size 214.11 KB)
[2023-06-07 11:19:54 #17 WARNING] Entering SLOW MODE. You have 2.0 GB available, but the engine would like at least 0.5 GB per core, i.e. 4.0 GB: https://github.com/gem/oq-engine/blob/master/doc/faq.md
[2023-06-07 11:19:54 #17 INFO] Extracting the hazard sites from the site model
/Users/chago/openquake/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column geology - only the converter will be used.
  return func(*args, 

  id | name
  65 | Events
  66 | Full Report
  67 | Ground Motion Fields
  68 | Realizations


[2023-06-07 11:19:55 #17 INFO] Exposing the outputs to the database
[2023-06-07 11:19:55 #17 INFO] Stored 591.73 KB on /Users/chago/oqdata/calc_17.hdf5 in 0 seconds
[2023-06-07 11:19:56 #18 INFO] chago@MacBook-Pro-de-Santiago.local running /Users/chago/GEM/ecd/Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/job_stations_none.ini [--hc=None]
[2023-06-07 11:19:56 #18 INFO] Using engine version 3.17.0-git60e4ac41e1
[2023-06-07 11:19:57 #18 WARNING] Using 8 cores on MacBook-Pro-de-Santiago.local
[2023-06-07 11:19:57 #18 INFO] Running EventBasedCalculator with concurrent_tasks = 16
[2023-06-07 11:19:57 #18 INFO] Checksum of the inputs: 2495010318 (total size 214.11 KB)
[2023-06-07 11:19:57 #18 WARNING] Entering SLOW MODE. You have 2.0 GB available, but the engine would like at least 0.5 GB per core, i.e. 4.0 GB: https://github.com/gem/oq-engine/blob/master/doc/faq.md
[2023-06-07 11:19:57 #18 INFO] Extracting the hazard sites from the site model
/Users/chago/openquake/lib/python3.9/site-packages

  id | name
  69 | Events
  70 | Full Report
  71 | Ground Motion Fields
  72 | Realizations
File saved to ../Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_13.csv
File saved to ../Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_14.csv
File saved to ../Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_15.csv
File saved to ../Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_16.csv
File saved to ../Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_17.csv
File saved to ../Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_18.csv


### Stations: Seismic

In [8]:
# Job.ini to be use in the sensitivity analysis
job_ini = os.path.join(oq_folder, 'job_stations_seismic.ini')

# Run OpenQuake calculations
jobs = oq.main(job_ini, oq_rups, oq_gmlt, concurrent_jobs=1)

# For each calculation, save gmfs in csv (only for PGA)
for job in jobs:
    calc_id = job.calc_id
    df, txt = oq_get_gmfs(calc_id)
    out_df = write_gmf_csv(df, 'PGA', comment=txt, 
                           out_path=oq_sens, suffix=calc_id)

[2023-06-07 11:20:24 #19 INFO] chago@MacBook-Pro-de-Santiago.local running /Users/chago/GEM/ecd/Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/job_stations_seismic.ini [--hc=None]
[2023-06-07 11:20:24 #19 INFO] Using engine version 3.17.0-git60e4ac41e1
[2023-06-07 11:20:25 #19 WARNING] Using 8 cores on MacBook-Pro-de-Santiago.local
[2023-06-07 11:20:25 #19 INFO] Running EventBasedCalculator with concurrent_tasks = 16
[2023-06-07 11:20:25 #19 INFO] Checksum of the inputs: 3707245561 (total size 225.87 KB)
[2023-06-07 11:20:25 #19 WARNING] Entering SLOW MODE. You have 1.9 GB available, but the engine would like at least 0.5 GB per core, i.e. 4.0 GB: https://github.com/gem/oq-engine/blob/master/doc/faq.md
[2023-06-07 11:20:25 #19 INFO] Extracting the hazard sites from the site model
/Users/chago/openquake/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column geology - only the converter will be used.
  return func(*arg

  id | name
  73 | Events
  74 | Full Report
  75 | Ground Motion Fields
  76 | Realizations


[2023-06-07 11:20:36 #20 INFO] chago@MacBook-Pro-de-Santiago.local running /Users/chago/GEM/ecd/Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/job_stations_seismic.ini [--hc=None]
[2023-06-07 11:20:36 #20 INFO] Using engine version 3.17.0-git60e4ac41e1
[2023-06-07 11:20:36 #20 WARNING] Using 8 cores on MacBook-Pro-de-Santiago.local
[2023-06-07 11:20:36 #20 INFO] Running EventBasedCalculator with concurrent_tasks = 16
[2023-06-07 11:20:36 #20 INFO] Checksum of the inputs: 866196689 (total size 225.91 KB)
[2023-06-07 11:20:36 #20 WARNING] Entering SLOW MODE. You have 1.9 GB available, but the engine would like at least 0.5 GB per core, i.e. 4.0 GB: https://github.com/gem/oq-engine/blob/master/doc/faq.md
[2023-06-07 11:20:36 #20 INFO] Extracting the hazard sites from the site model
/Users/chago/openquake/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column geology - only the converter will be used.
  return func(*args

  id | name
  77 | Events
  78 | Full Report
  79 | Ground Motion Fields
  80 | Realizations


[2023-06-07 11:20:45 #21 INFO] chago@MacBook-Pro-de-Santiago.local running /Users/chago/GEM/ecd/Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/job_stations_seismic.ini [--hc=None]
[2023-06-07 11:20:45 #21 INFO] Using engine version 3.17.0-git60e4ac41e1
[2023-06-07 11:20:46 #21 WARNING] Using 8 cores on MacBook-Pro-de-Santiago.local
[2023-06-07 11:20:46 #21 INFO] Running EventBasedCalculator with concurrent_tasks = 16
[2023-06-07 11:20:46 #21 INFO] Checksum of the inputs: 598481994 (total size 225.91 KB)
[2023-06-07 11:20:46 #21 WARNING] Entering SLOW MODE. You have 2.0 GB available, but the engine would like at least 0.5 GB per core, i.e. 4.0 GB: https://github.com/gem/oq-engine/blob/master/doc/faq.md
[2023-06-07 11:20:46 #21 INFO] Extracting the hazard sites from the site model
/Users/chago/openquake/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column geology - only the converter will be used.
  return func(*args

...

  id | name
  85 | Events
  86 | Full Report
  87 | Ground Motion Fields
  88 | Realizations


[2023-06-07 11:21:03 #23 INFO] chago@MacBook-Pro-de-Santiago.local running /Users/chago/GEM/ecd/Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/job_stations_seismic.ini [--hc=None]
[2023-06-07 11:21:03 #23 INFO] Using engine version 3.17.0-git60e4ac41e1
[2023-06-07 11:21:03 #23 WARNING] Using 8 cores on MacBook-Pro-de-Santiago.local
[2023-06-07 11:21:03 #23 INFO] Running EventBasedCalculator with concurrent_tasks = 16
[2023-06-07 11:21:03 #23 INFO] Checksum of the inputs: 1521815449 (total size 224.83 KB)
[2023-06-07 11:21:03 #23 WARNING] Entering SLOW MODE. You have 2.1 GB available, but the engine would like at least 0.5 GB per core, i.e. 4.0 GB: https://github.com/gem/oq-engine/blob/master/doc/faq.md
[2023-06-07 11:21:03 #23 INFO] Extracting the hazard sites from the site model
/Users/chago/openquake/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column geology - only the converter will be used.
  return func(*arg

  id | name
  89 | Events
  90 | Full Report
  91 | Ground Motion Fields
  92 | Realizations


[2023-06-07 11:21:10 #24 INFO] chago@MacBook-Pro-de-Santiago.local running /Users/chago/GEM/ecd/Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/job_stations_seismic.ini [--hc=None]
[2023-06-07 11:21:10 #24 INFO] Using engine version 3.17.0-git60e4ac41e1
[2023-06-07 11:21:10 #24 WARNING] Using 8 cores on MacBook-Pro-de-Santiago.local
[2023-06-07 11:21:10 #24 INFO] Running EventBasedCalculator with concurrent_tasks = 16
[2023-06-07 11:21:10 #24 INFO] Checksum of the inputs: 2486701842 (total size 224.83 KB)
[2023-06-07 11:21:10 #24 WARNING] Entering SLOW MODE. You have 2.0 GB available, but the engine would like at least 0.5 GB per core, i.e. 4.0 GB: https://github.com/gem/oq-engine/blob/master/doc/faq.md
[2023-06-07 11:21:10 #24 INFO] Extracting the hazard sites from the site model
/Users/chago/openquake/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column geology - only the converter will be used.
  return func(*arg

  id | name
  93 | Events
  94 | Full Report
  95 | Ground Motion Fields
  96 | Realizations
File saved to ../Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_19.csv
File saved to ../Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_20.csv
File saved to ../Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_21.csv
File saved to ../Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_22.csv
File saved to ../Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_23.csv
File saved to ../Greece/20151117_M6.5_Lefkada/OpenQuake_gmfs/Sensitivity/gmf_median_PGA_24.csv


### Stations: All (i.e., seismic and macroseismic stations)

In [ ]:
# # Job.ini to be use in the sensitivity analysis
# job_ini = os.path.join(oq_folder, 'job_stations_all.ini')

# # Run OpenQuake calculations
# jobs = oq.main(job_ini, oq_rups, oq_gmlt, concurrent_jobs=1)

# # For each calculation, save gmfs in csv (only for PGA)
# for job in jobs:
#     calc_id = job.calc_id
#     df, txt = oq_get_gmfs(calc_id)
#     out_df = write_gmf_csv(df, 'PGA', comment=txt, 
#                            out_path=oq_sens, suffix=calc_id)